In [1]:
import pandas as pd
import numpy as np






### Popularity-Based Recommender 

In [2]:
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
tags.head(25)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [5]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
df = (
ratings.groupby('movieId')
       .agg({'userId':['count'], 'rating':['mean']})
)
df.columns=['count', 'rating_mean']
df.sort_values(by=['count'], ascending=False)

,count,rating_mean
movieId,,
356,329,4.164134
318,317,4.429022
296,307,4.197068
593,279,4.161290
2571,278,4.192446
...,...,...
4093,1,1.500000
4089,1,2.000000
58351,1,4.000000


### Content-Based Recommender

In [7]:
def cont_recomm(movieId):
    
    tag_movieId = []
    
    for tag in tags[tags['movieId'] == movieId].tag.tolist():
        for lst in list(tags[tags['tag'] == tag].movieId):
            tag_movieId.append(lst)
    
    movies_name = []
    
    for movieID in tag_movieId:
        for movie_genre in list(movies[movies['movieId'] == movieId].genres)[0].split('|'):
            if movie_genre in list(movies[movies['movieId'] == movieID].genres)[0].split('|'):
                movies_name.append(movies[movies['movieId'] == movieID])
        
    return pd.concat(movies_name).drop_duplicates().head(20)

In [8]:
cont_recomm(106782)

,movieId,title,genres
8305,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
3016,4034,Traffic (2000),Crime|Drama|Thriller
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
5610,27020,Gia (1998),Drama|Romance
1430,1953,"French Connection, The (1971)",Action|Crime|Thriller
4168,6002,Love Liza (2002),Drama
5266,8645,"Maria Full of Grace (Maria, Llena eres de grac...",Crime|Drama
6676,57669,In Bruges (2008),Comedy|Crime|Drama|Thriller
1298,1732,"Big Lebowski, The (1998)",Comedy|Crime
6358,49530,Blood Diamond (2006),Action|Adventure|Crime|Drama|Thriller|War







### Pearson Item-Based Recommender 

In [9]:
movie_item = (pd.pivot_table(data=ratings, values='rating', 
                             index='userId', columns='movieId'))
movie_item

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def pearson_item_recomm(movieId_num, nom):
    
    pearson_similar = pd.DataFrame(movie_item.loc[:, list(set(movie_item.columns) - set([movieId_num]))]\
                        .corrwith(movie_item[movieId_num]))\
                        .rename(columns={0:'PearsonCoeff'})\
                        .sort_values(by=['PearsonCoeff'], ascending=False)\
                        .reset_index()
    
    pearson_similar = pearson_similar[pearson_similar['PearsonCoeff'] >= 0.5]
    
    from itertools import combinations
    
    movies_name = []
    
    for movieID in list(pearson_similar.movieId):
        if len(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|')) == 1:
            for movie_genre in list(movies[movies['movieId'] == movieId_num].genres)[0].split('|'):
                if movie_genre in list(movies[movies['movieId'] == movieID].genres)[0].split('|'):
                    movies_name.append(movies[movies['movieId'] == movieID])
        elif len(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|')) > 1:
            for movie_genre in list(combinations(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|'), 2)):
                if (movie_genre[0] in list(movies[movies['movieId'] == movieID].genres)[0].split('|'))\
                & (movie_genre[1] in list(movies[movies['movieId'] == movieID].genres)[0].split('|')):
                    movies_name.append(movies[movies['movieId'] == movieID])
    
    return pd.concat(movies_name).drop_duplicates().head(nom)

In [11]:
movies[movies['movieId'] == 2857]

,movieId,title,genres
2144,2857,Yellow Submarine (1968),Adventure|Animation|Comedy|Fantasy|Musical


In [12]:
pearson_item_recomm(2857, 50)

C:\Users\Privat\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\Privat\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,movieId,title,genres
6729,59022,Harold & Kumar Escape from Guantanamo Bay (2008),Adventure|Comedy
2445,3258,Death Becomes Her (1992),Comedy|Fantasy
1556,2093,Return to Oz (1985),Adventure|Children|Fantasy
3093,4153,Down to Earth (2001),Comedy|Fantasy|Romance
732,952,Around the World in 80 Days (1956),Adventure|Comedy
3069,4121,Innerspace (1987),Action|Adventure|Comedy|Sci-Fi
4841,7228,Cool World (1992),Animation|Comedy|Fantasy
2138,2846,"Adventures of Milo and Otis, The (Koneko monog...",Adventure|Children|Comedy|Drama
4789,7132,"Night at the Opera, A (1935)",Comedy|Musical|Romance
6149,44022,Ice Age 2: The Meltdown (2006),Adventure|Animation|Children|Comedy


### Cosine_Similiarity-Based Recommender

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
movie_itemm = (pd.pivot_table(data=ratings, values='rating', 
                             index='userId', columns='movieId', fill_value=0))
movie_itemm
movie_similiarity_matrix = pd.DataFrame(cosine_similarity(movie_itemm.T), columns=movie_itemm.columns, index=movie_itemm.columns)
movie_similiarity_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.410562,0.296917,0.035573,0.308762,0.376316,0.277491,0.131629,0.232586,0.395573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.410562,1.000000,0.282438,0.106415,0.287795,0.297009,0.228576,0.172498,0.044835,0.417693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.296917,0.282438,1.000000,0.092406,0.417802,0.284257,0.402831,0.313434,0.304840,0.242954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.035573,0.106415,0.092406,1.000000,0.188376,0.089685,0.275035,0.158022,0.000000,0.095598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.308762,0.287795,0.417802,0.188376,1.000000,0.298969,0.474002,0.283523,0.335058,0.218061,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193583,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [14]:
movie_similiarity_matrix[19].reset_index().rename(columns={19:'Cosine_corr'}).sort_values(by='Cosine_corr', ascending=False).head(25)

,movieId,Cosine_corr
18,19,1.000000
302,344,0.604960
325,367,0.514114
197,231,0.502934
1,2,0.497560
322,364,0.483764
436,500,0.479588
138,165,0.474048
483,551,0.470248
217,253,0.467595


In [15]:
def cosine_recomm(movieId_num, nom):
    
    from sklearn.metrics.pairwise import cosine_similarity
    movie_itemm = (pd.pivot_table(data=ratings, values='rating', 
                             index='userId', columns='movieId', fill_value=0))
    movie_itemm
    movie_similiarity_matrix = pd.DataFrame(cosine_similarity(movie_itemm.T),\
                                            columns=movie_itemm.columns, index=movie_itemm.columns)
    
    movieId_matrix = movie_similiarity_matrix[movieId_num].reset_index()\
                                                          .rename(columns={movieId_num:'Cosine_corr'})\
                                                          .sort_values(by='Cosine_corr', ascending=False)
    
    movieId_matrix = movieId_matrix[movieId_matrix['Cosine_corr'] >= 0.3]
    
    from itertools import combinations
    
    
    movies_name = []
    
    for movieID in list(movieId_matrix.movieId):
        if len(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|')) == 1:
            for movie_genre in list(movies[movies['movieId'] == movieId_num].genres)[0].split('|'):
                if movie_genre in list(movies[movies['movieId'] == movieID].genres)[0].split('|'):
                    movies_name.append(movies[movies['movieId'] == movieID])
        elif len(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|')) > 1:
            for movie_genre in list(combinations(list(movies[movies['movieId'] == movieId_num].genres)[0].split('|'), 2)):
                if (movie_genre[0] in list(movies[movies['movieId'] == movieID].genres)[0].split('|'))\
                & (movie_genre[1] in list(movies[movies['movieId'] == movieID].genres)[0].split('|')):
                    movies_name.append(movies[movies['movieId'] == movieID])
                
    return pd.concat(movies_name).drop_duplicates().head(nom)

In [16]:
list(movies[movies['movieId'] == 79008].genres)[0].split('|')

['Animation', 'Comedy', 'Fantasy']

In [17]:
cosine_recomm(79008, 50)

,movieId,title,genres
7368,79008,South Park: Imaginationland (2008),Animation|Comedy|Fantasy
8339,107953,Dragon Ball Z: Battle of Gods (2013),Action|Animation|Fantasy|IMAX
6778,60161,Futurama: The Beast with a Billion Backs (2008),Action|Animation|Comedy|Romance|Sci-Fi
5637,27369,Daria: Is It Fall Yet? (2000),Animation|Comedy
7530,84637,Gnomeo & Juliet (2011),Adventure|Animation|Children|Comedy|Fantasy|Ro...
2984,3997,Dungeons & Dragons (2000),Action|Adventure|Comedy|Fantasy
8148,102084,Justice League: Doom (2012),Action|Animation|Fantasy
7871,94160,Dante's Inferno: An Animated Epic (2010),Action|Animation|Fantasy
7122,71129,Green Lantern: First Flight (2009),Action|Adventure|Animation|Fantasy|Sci-Fi
5236,8580,Into the Woods (1991),Adventure|Comedy|Fantasy|Musical


### User-Based Prediction Recommender

In [18]:
movie_itemm = (pd.pivot_table(data=ratings, values='rating', 
                             index='userId', columns='movieId', fill_value=0))
movie_itemm

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0,0.0,4.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0


In [19]:
def user_pred_recomm(userId):
    
    from sklearn.metrics.pairwise import cosine_similarity
    
    user_similiarity_matrix = pd.DataFrame(cosine_similarity(movie_itemm), columns=movie_itemm.index, index=movie_itemm.index)
    user_similiarity_matrix
    
    userId_corr_weights = user_similiarity_matrix[userId].drop([userId], axis=0)/(user_similiarity_matrix[userId].drop([userId], axis=0).sum())
    
    userId_row = movie_itemm[movie_itemm.index == userId]
    
    user_movie_detail = []
    
    for movieId in list(userId_row.columns):
        if list(userId_row[movieId])[0] == 0:
            user_movie_detail.append([movieId, movie_itemm[movieId].drop([userId], axis=0).dot(userId_corr_weights)])
            
    recomm_movies = pd.DataFrame(user_movie_detail, columns=['movieId', 'pred_rating'])\
                      .sort_values('pred_rating', ascending=False)\
                      .merge(movies, how='left', on='movieId')
    
    return recomm_movies[recomm_movies.pred_rating >= 1.5]

In [20]:
user_pred_recomm(60)

,movieId,pred_rating,title,genres
0,296,2.614973,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,356,2.581781,Forrest Gump (1994),Comedy|Drama|Romance|War
2,593,2.300622,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
3,2571,2.213145,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4,2959,1.997208,Fight Club (1999),Action|Crime|Drama|Thriller
5,260,1.925855,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
6,110,1.792704,Braveheart (1995),Action|Drama|War
7,1196,1.618827,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
8,47,1.584842,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
9,2858,1.578990,American Beauty (1999),Drama|Romance


### Recommender Evaluation

In [24]:
rating_pos = pd.DataFrame(np.nonzero(np.array(movie_itemm))).T.rename(columns={0:'row', 1:'column'})
rating_pos

,row,column
0,0,0
1,0,2
2,0,5
3,0,43
4,0,46
...,...,...
100831,609,9416
100832,609,9443
100833,609,9444
100834,609,9445


In [81]:
from sklearn.model_selection import train_test_split

train_pos, test_pos = train_test_split(rating_pos, random_state=1023, test_size=.1)

In [82]:
train_pos

,row,column
46012,304,882
70778,451,2133
87814,566,958
75147,473,6272
61466,404,2224
...,...,...
94871,598,7887
93372,598,1738
22747,155,658
31267,216,1609


In [83]:
train_pos.values

array([[ 304,  882],
       [ 451, 2133],
       [ 566,  958],
       ...,
       [ 155,  658],
       [ 216, 1609],
       [ 216, 1781]], dtype=int64)

In [84]:
train = np.zeros(movie_itemm.shape)
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [85]:
for pos in train_pos.values:
    train[pos[0], pos[1]] = movie_itemm.iloc[pos[0], pos[1]]
    
train_part = pd.DataFrame(train, columns=movie_itemm.columns, index=movie_itemm.index)
train_part

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
test = np.zeros(movie_itemm.shape)

for pos in test_pos.values:
    test[pos[0], pos[1]] = movie_itemm.iloc[pos[0], pos[1]]
    
test_part = pd.DataFrame(test, columns=movie_itemm.columns, index=movie_itemm.index)
test_part

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
test_pos['true_pred'] = test_pos.apply(lambda row: movie_itemm.iloc[row[0], row[1]], axis=1)
test_pos

,row,column,true_pred
52968,347,3617,5.0
63211,413,2042,2.0
85303,554,79,4.0
95076,598,9471,4.5
6357,42,239,5.0
...,...,...,...
84850,550,3868,4.0
65075,414,5834,4.0
78456,487,4421,4.0
88979,573,126,3.0


In [138]:
def userpred_recomm(userId, movieID):
    
    from sklearn.metrics.pairwise import cosine_similarity
    
    user_similiarity_matrix = pd.DataFrame(cosine_similarity(train_part), columns=movie_itemm.index, index=movie_itemm.index)
    user_similiarity_matrix
    
    userId_corr_weights = user_similiarity_matrix[userId].drop([userId], axis=0)/(user_similiarity_matrix[userId].drop([userId], axis=0).sum())
    
    userId_row = train_part[train_part.index == userId]
    
    user_movie_detail = []
    
    for movieeId in list(userId_row.columns):
        if list(userId_row[movieeId])[0] == 0:
            user_movie_detail.append([movieeId, train_part[movieeId].drop([userId], axis=0).dot(userId_corr_weights)])
            
    recomm_movies = pd.DataFrame(user_movie_detail, columns=['movieId', 'pred_rating'])
                      #.merge(movies, how='left', on='movieId')
    
    pos_index = pd.DataFrame(train_part.columns)
    movieIDD = pos_index.loc[movieID, 'movieId']
    
    return recomm_movies#list(recomm_movies[recomm_movies.movieId == movieIDD].pred_rating)

In [134]:
test_pos_trial = test_pos[0:50]
test_pos_trial['predt_rating'] = test_pos_trial.apply(lambda row: userpred_recomm(row['row'], row['column']), axis=1)
test_pos_trial

C:\Users\Privat\AppData\Local\Temp\ipykernel_1344\3904495804.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pos_trial['predt_rating'] = test_pos_trial.apply(lambda row: userpred_recomm(row['row'], row['column']), axis=1)


,row,column,true_pred,predt_rating
52968,347,3617,5.0,[0.5050831823642408]
63211,413,2042,2.0,[0.16125515618729538]
85303,554,79,4.0,[0.08407702846633286]
95076,598,9471,4.5,[0.010077127561554758]
6357,42,239,5.0,[0.2484524509766678]
88307,569,938,3.5,[0.7127119232533534]
57677,379,5977,4.0,[0.0184315485591616]
32401,220,1429,3.0,[0.29464794500987534]
49183,317,2390,4.5,[0.25453034000787866]
47923,308,5725,4.0,[0.24398733856608992]


In [139]:
userpred_recomm(18, 549)

,movieId,pred_rating
0,3,0.237561
1,4,0.016513
2,5,0.209163
3,7,0.220405
4,8,0.039065
...,...,...
9274,193581,0.005716
9275,193583,0.005002
9276,193585,0.005002
9277,193587,0.005002


In [136]:
pos_index = pd.DataFrame(train_part.columns)
movieIDD = pos_index.loc[549, 'movieId']
movieIDD

653

In [140]:
train_part[train_part.index == 18][[653]]

movieId,653
userId,
18,3.0


In [142]:
train_part.loc[18, 653]

3.0